## Individual runs from primary analysis
This notebook illustrates the trajectories of individual runs from the candidate ('mob') analysis.

In [ ]:
import pandas as pd
import numpy as np
from emutools.tex import DummyTexDoc
import arviz as az
from aust_covid.constants import RUN_IDS, BURN_IN, set_project_base_path
from aust_covid.calibration import get_targets
from emutools.calibration import plot_spaghetti, plot_param_hover_spaghetti
from aust_covid.plotting import plot_cdr_examples, plot_3d_spaghetti
import warnings
warnings.simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

project_paths = set_project_base_path("../")
RUNS_PATH = project_paths["RUNS_PATH"]

In [ ]:
dummy_doc = DummyTexDoc()
targets = get_targets(dummy_doc, cross_ref=False)
spaghetti = pd.read_hdf(RUNS_PATH / RUN_IDS["mob"] / "output/results.hdf", "spaghetti")

In [ ]:
plot_spaghetti(spaghetti, ["notifications_ma", "deaths_ma", "adult_seropos_prop", "reproduction_number"], 2, targets)

In [ ]:
idata = az.from_netcdf(RUNS_PATH / RUN_IDS["mob"] / "output/calib_full_out.nc")
burnt_idata = idata.sel(draw=np.s_[BURN_IN:])
n_samples = 12
sampled_idata = az.extract(burnt_idata, num_samples=n_samples)
plot_cdr_examples(sampled_idata.variables["start_cdr"])

In [ ]:
plot_param_hover_spaghetti(spaghetti["notifications_ma"], idata)

In [ ]:
plot_3d_spaghetti("adult_seropos_prop", spaghetti, targets)